In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [2]:
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [3]:
data.head()

,id,label,date,f1,f2,f3,f4,f5,f6,f7,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,...,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,...,302.0,324.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0
2,1270cb8a85eedd57672b2c6297fa5633e36773a2c3a351...,0,20171022,0,0,1,0,100102.0,0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9fa009724ee7ff9d688ae321304fbc78f608cdabbfdd2b...,0,20171029,0,0,0,1,100807.0,1,4,...,302.0,322.0,341.0,57.0,251.0,175.0,176.0,176.0,49.0,150.0
4,1da482485d7e8bcefae7e9d0d1167cec3ac111cfa71d8b...,0,20171002,1,1,0,1,100805.0,1,5,...,302.0,301.0,301.0,74.0,302.0,182.0,181.0,182.0,51.0,181.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994731 entries, 0 to 994730
Columns: 300 entries, id to f297
dtypes: float64(279), int64(20), object(1)
memory usage: 2.2+ GB


In [5]:
data['label'].value_counts()   #label取值含义：1是有风险，0是无风险，-1是无标签，需要参赛者自行判断处理。

 0    977884
 1     12122
-1      4725
Name: label, dtype: int64

In [6]:
data.describe()

,label,date,f1,f2,f3,f4,f5,f6,f7,f8,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
count,994731.000000,9.947310e+05,994731.000000,994731.000000,994731.000000,994731.000000,794906.000000,994731.000000,994731.000000,994731.000000,...,787146.000000,787146.000000,787146.000000,787146.000000,787146.000000,787146.000000,787146.000000,787146.000000,787146.000000,787146.000000
mean,0.007436,2.017098e+07,0.726283,0.500948,0.699285,0.525461,100652.575330,0.701303,4.201802,1.404783,...,159.532191,164.423796,174.171849,57.639792,149.509402,105.056598,105.243853,105.246376,41.244121,99.023500
std,0.129927,6.011735e+01,0.651947,0.500363,0.640534,0.523872,4023.235737,0.757799,1.922035,0.579449,...,149.126719,152.228807,164.144148,63.122004,142.628265,100.003140,100.155753,100.153540,45.658241,96.240483
min,-1.000000,2.017090e+07,0.000000,0.000000,0.000000,0.000000,10000.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,2.017092e+07,0.000000,0.000000,0.000000,0.000000,100802.000000,0.000000,3.000000,1.000000,...,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,0.000000,2.017101e+07,1.000000,1.000000,1.000000,1.000000,100804.000000,1.000000,4.000000,1.000000,...,301.000000,301.000000,301.000000,26.000000,184.000000,164.000000,164.000000,164.000000,17.000000,115.000000
75%,0.000000,2.017102e+07,1.000000,1.000000,1.000000,1.000000,100806.000000,1.000000,6.000000,2.000000,...,302.000000,304.000000,306.000000,109.000000,301.000000,199.000000,199.000000,199.000000,79.000000,194.000000
max,1.000000,2.017110e+07,2.000000,2.000000,2.000000,2.000000,150406.000000,4.000000,7.000000,2.000000,...,302.000000,388.000000,669.000000,302.000000,302.000000,299.000000,299.000000,299.000000,280.000000,299.000000


In [7]:
# %matplotlib inline   

# data.hist(bins=50, figsize=(30,30))
# plt.show()
pd.set_option('display.max_rows', None) 
data.loc[2, :]

id       1270cb8a85eedd57672b2c6297fa5633e36773a2c3a351...
label                                                    0
date                                              20171022
f1                                                       0
f2                                                       0
f3                                                       1
f4                                                       0
f5                                                  100102
f6                                                       0
f7                                                       6
f8                                                       1
f9                                                       0
f10                                                      0
f11                                                      0
f12                                                      2
f13                                                      1
f14                                                     

In [8]:
data.loc[1, :]

id       d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...
label                                                    0
date                                              20170917
f1                                                       0
f2                                                       1
f3                                                       1
f4                                                       1
f5                                                  100805
f6                                                       1
f7                                                       5
f8                                                       2
f9                                                       1
f10                                                      1
f11                                                      0
f12                                                      2
f13                                                      1
f14                                                     

In [14]:
data['f5'].value_counts() 

100805.0    185834
100806.0    165376
100803.0    105475
100802.0     86924
100804.0     82597
100807.0     33191
100801.0     17694
100102.0     13033
100103.0     12811
100800.0      7354
100610.0      6575
100608.0      5747
100609.0      5501
100607.0      5142
100100.0      5138
100101.0      5099
100704.0      4486
100020.0      3132
100106.0      3103
100105.0      3029
100021.0      2672
100705.0      2610
100329.0      2418
100328.0      2363
100703.0      1928
100701.0      1799
100611.0      1521
100325.0      1436
100702.0      1366
100326.0      1181
100019.0       857
100311.0       821
100700.0       690
100320.0       670
100321.0       659
100310.0       638
100016.0       602
100015.0       497
150203.0       489
100018.0       471
150202.0       434
100322.0       397
100323.0       387
50201.0        384
50200.0        383
100009.0       339
100312.0       326
100306.0       319
100305.0       306
100324.0       296
100605.0       276
50101.0        253
100319.0    

In [8]:
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "fillna.csv")
data = pd.read_csv(csv_path)

In [3]:
data.head()

,id,label,date,f1,f2,f3,f4,f5,f6,f7,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,...,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,...,302.0,324.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0
2,1270cb8a85eedd57672b2c6297fa5633e36773a2c3a351...,0,20171022,0,0,1,0,100102.0,0,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9fa009724ee7ff9d688ae321304fbc78f608cdabbfdd2b...,0,20171029,0,0,0,1,100807.0,1,4,...,302.0,322.0,341.0,57.0,251.0,175.0,176.0,176.0,49.0,150.0
4,1da482485d7e8bcefae7e9d0d1167cec3ac111cfa71d8b...,0,20171002,1,1,0,1,100805.0,1,5,...,302.0,301.0,301.0,74.0,302.0,182.0,181.0,182.0,51.0,181.0


In [6]:
data.describe()

,label,date,f1,f2,f3,f4,f5,f6,f7,f8,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
count,994731.000000,9.947310e+05,994731.000000,994731.000000,994731.000000,994731.000000,994731.00000,994731.000000,994731.000000,994731.000000,...,994731.000000,994731.000000,994731.000000,994731.000000,994731.000000,994731.000000,994731.000000,994731.000000,994731.000000,994731.000000
mean,0.007436,2.017098e+07,0.726283,0.500948,0.699285,0.525461,80433.13825,0.701303,4.201802,1.404783,...,126.240286,130.111088,137.824873,45.611258,118.309099,83.132908,83.281086,83.283083,32.637110,78.358825
std,0.129927,6.011735e+01,0.651947,0.500363,0.640534,0.523872,40487.64318,0.757799,1.922035,0.579449,...,147.650492,151.003637,162.265730,60.840312,140.672913,98.672343,98.827660,98.826329,43.937949,94.597069
min,-1.000000,2.017090e+07,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.017092e+07,0.000000,0.000000,0.000000,0.000000,100103.00000,0.000000,3.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,0.000000,2.017101e+07,1.000000,1.000000,1.000000,1.000000,100803.00000,1.000000,4.000000,1.000000,...,3.000000,5.000000,6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,0.000000,2.017102e+07,1.000000,1.000000,1.000000,1.000000,100805.00000,1.000000,6.000000,2.000000,...,301.000000,302.000000,302.000000,92.000000,301.000000,193.000000,193.000000,193.000000,65.000000,186.000000
max,1.000000,2.017110e+07,2.000000,2.000000,2.000000,2.000000,150406.00000,4.000000,7.000000,2.000000,...,302.000000,388.000000,669.000000,302.000000,302.000000,299.000000,299.000000,299.000000,280.000000,299.000000


In [15]:
X = X_df.as_matrix()
y = y_df.as_matrix()

In [16]:
print(X.shape)
print(y.shape)

(994731, 297)
(994731,)


In [3]:
data = data[data['label'] != -1]

In [4]:
X_df = data.loc[:,'date':'f297']
y_df = data.loc[:,'label']
X = X_df.as_matrix()
y = y_df.as_matrix()
print(X.shape)
print(y.shape)

(990006, 298)
(990006,)


In [5]:
X[:5]

array([[2.0171103e+07, 0.0000000e+00, 0.0000000e+00, ..., 2.0300000e+02,
        6.1000000e+01, 2.0100000e+02],
       [2.0170917e+07, 0.0000000e+00, 1.0000000e+00, ..., 1.6100000e+02,
        8.0000000e+00, 1.6000000e+02],
       [2.0171022e+07, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [2.0171029e+07, 0.0000000e+00, 0.0000000e+00, ..., 1.7600000e+02,
        4.9000000e+01, 1.5000000e+02],
       [2.0171002e+07, 1.0000000e+00, 1.0000000e+00, ..., 1.8200000e+02,
        5.1000000e+01, 1.8100000e+02]])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [8]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [13]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

[1]	valid_0's l2: 0.226395	valid_0's auc: 0.922407
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.20513	valid_0's auc: 0.952069
[3]	valid_0's l2: 0.185956	valid_0's auc: 0.954
[4]	valid_0's l2: 0.168678	valid_0's auc: 0.953753
[5]	valid_0's l2: 0.153109	valid_0's auc: 0.957756
[6]	valid_0's l2: 0.139039	valid_0's auc: 0.957838
[7]	valid_0's l2: 0.12636	valid_0's auc: 0.96075
[8]	valid_0's l2: 0.114896	valid_0's auc: 0.960181
[9]	valid_0's l2: 0.104543	valid_0's auc: 0.960447
[10]	valid_0's l2: 0.0951967	valid_0's auc: 0.961071
[11]	valid_0's l2: 0.0867438	valid_0's auc: 0.961804
[12]	valid_0's l2: 0.079107	valid_0's auc: 0.961817
[13]	valid_0's l2: 0.0722094	valid_0's auc: 0.961743
[14]	valid_0's l2: 0.0659717	valid_0's auc: 0.961665
[15]	valid_0's l2: 0.0603465	valid_0's auc: 0.963454
[16]	valid_0's l2: 0.0552453	valid_0's auc: 0.963628
[17]	valid_0's l2: 0.0506449	valid_0's auc: 0.963075
[18]	valid_0's l2: 0.0464749	valid_0's auc: 0.963711
[19]	valid

[153]	valid_0's l2: 0.00599058	valid_0's auc: 0.987458
[154]	valid_0's l2: 0.00598994	valid_0's auc: 0.987462
[155]	valid_0's l2: 0.00598453	valid_0's auc: 0.987493
[156]	valid_0's l2: 0.00598099	valid_0's auc: 0.987534
[157]	valid_0's l2: 0.005976	valid_0's auc: 0.987576
[158]	valid_0's l2: 0.00597345	valid_0's auc: 0.987594
[159]	valid_0's l2: 0.00596611	valid_0's auc: 0.987679
[160]	valid_0's l2: 0.00596292	valid_0's auc: 0.987704
[161]	valid_0's l2: 0.00595727	valid_0's auc: 0.98774
[162]	valid_0's l2: 0.00595737	valid_0's auc: 0.987799
[163]	valid_0's l2: 0.00595217	valid_0's auc: 0.98784
[164]	valid_0's l2: 0.00595309	valid_0's auc: 0.98785
[165]	valid_0's l2: 0.00594807	valid_0's auc: 0.987871
[166]	valid_0's l2: 0.00594458	valid_0's auc: 0.987903
[167]	valid_0's l2: 0.00594186	valid_0's auc: 0.98791
[168]	valid_0's l2: 0.00593608	valid_0's auc: 0.98795
[169]	valid_0's l2: 0.00593426	valid_0's auc: 0.987957
[170]	valid_0's l2: 0.00592991	valid_0's auc: 0.987978
[171]	valid_0's l

In [14]:
gbm.save_model('model.txt')

In [15]:
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_a.csv")
test_data = pd.read_csv(csv_path)
test_data.head()

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,8e8290c270ec4bc3448dd5edd35c6f059b42d38f9ddd6f...,20180120,0,0,1,1,100809.0,1,3,2,...,302.0,302.0,302.0,134.0,302.0,215.0,215.0,215.0,94.0,214.0
1,f36c5fa5c0e7afccf733a4d74c7e06ffe43cc8fd24eda7...,20180105,1,1,0,0,100808.0,1,5,2,...,39.0,40.0,40.0,39.0,40.0,39.0,39.0,39.0,39.0,39.0
2,ee85f808b1fd49eaba308527e1686c509dc8e3e5057488...,20180109,0,0,0,0,100803.0,0,6,1,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
3,0df905aa187938d56a9b0816b13f54ac1f87d658a33cd0...,20180118,1,1,1,1,100808.0,0,3,2,...,301.0,301.0,302.0,122.0,301.0,208.0,208.0,207.0,89.0,207.0
4,15f532f979c4f092bbbe28e5409c8c3b8454ece8f1ab3e...,20180109,1,1,1,1,100809.0,0,2,2,...,301.0,302.0,302.0,118.0,302.0,251.0,251.0,251.0,101.0,251.0


In [16]:
test_data = test_data.fillna(0)

In [17]:
X_df_pred = test_data.loc[:,'date':'f297']
X_pred = X_df_pred.as_matrix()
X_pred.shape

(491668, 298)

In [40]:
test_data.describe()

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
count,4.916680e+05,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,...,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000,491668.000000
mean,2.018013e+07,0.770091,0.500277,0.749955,0.540977,75543.415669,0.828333,4.005492,1.391703,0.621680,...,127.076806,128.190041,129.217136,40.985476,113.508624,91.112735,91.206190,91.206161,31.974536,82.289394
std,3.109189e+01,0.668126,0.500033,0.662133,0.537011,43810.703649,0.941432,1.886214,0.588867,0.597478,...,147.878316,147.855822,148.021180,57.898143,136.483789,107.789742,107.881654,107.880933,47.168140,100.853719
min,2.018010e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.018011e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,2.018012e+07,1.000000,1.000000,1.000000,1.000000,100808.000000,1.000000,4.000000,1.000000,1.000000,...,3.000000,4.000000,6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.018013e+07,1.000000,1.000000,1.000000,1.000000,100809.000000,1.000000,6.000000,2.000000,1.000000,...,301.000000,302.000000,302.000000,81.000000,301.000000,207.000000,207.000000,207.000000,61.000000,195.000000
max,2.018020e+07,2.000000,2.000000,2.000000,2.000000,150501.000000,4.000000,7.000000,2.000000,2.000000,...,302.000000,312.000000,322.000000,302.000000,302.000000,302.000000,302.000000,302.000000,301.000000,302.000000


In [41]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491668 entries, 0 to 491667
Columns: 299 entries, id to f297
dtypes: float64(279), int64(19), object(1)
memory usage: 1.1+ GB


In [18]:
y_pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)

In [19]:
y_pred.shape

(491668,)

In [21]:
y_pred[30000:400000]

array([0.00049012, 0.00030498, 0.00026037, ..., 0.00131765, 0.0081077 ,
       0.00048634])

In [25]:
result = pd.DataFrame({'id':test_data['id'].as_matrix(), 'score':y_pred})
result.to_csv("result.csv", index=False)

In [9]:
######将-1看作有风险
data['label'].value_counts()

 0    977884
 1     12122
-1      4725
Name: label, dtype: int64

In [10]:
data.loc[data['label'] == -1, 'label'] = 1

In [11]:
data['label'].value_counts()

0    977884
1     16847
Name: label, dtype: int64

In [12]:
X_df = data.loc[:,'date':'f297']
y_df = data.loc[:,'label']
X = X_df.as_matrix()
y = y_df.as_matrix()
print(X.shape)
print(y.shape)

(994731, 298)
(994731,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [16]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=300,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

[1]	valid_0's auc: 0.929271	valid_0's l2: 0.226765
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.943974	valid_0's l2: 0.205859
[3]	valid_0's auc: 0.94454	valid_0's l2: 0.186989
[4]	valid_0's auc: 0.944225	valid_0's l2: 0.170002
[5]	valid_0's auc: 0.944859	valid_0's l2: 0.15465
[6]	valid_0's auc: 0.945026	valid_0's l2: 0.14082
[7]	valid_0's auc: 0.945307	valid_0's l2: 0.128353
[8]	valid_0's auc: 0.945436	valid_0's l2: 0.117078
[9]	valid_0's auc: 0.946287	valid_0's l2: 0.106868
[10]	valid_0's auc: 0.946498	valid_0's l2: 0.0976521
[11]	valid_0's auc: 0.950244	valid_0's l2: 0.0893513
[12]	valid_0's auc: 0.950393	valid_0's l2: 0.0818228
[13]	valid_0's auc: 0.951425	valid_0's l2: 0.0750267
[14]	valid_0's auc: 0.951494	valid_0's l2: 0.0688813
[15]	valid_0's auc: 0.951544	valid_0's l2: 0.063322
[16]	valid_0's auc: 0.951545	valid_0's l2: 0.0582985
[17]	valid_0's auc: 0.951675	valid_0's l2: 0.0537657
[18]	valid_0's auc: 0.954384	valid_0's l2: 0.0496489
[19]	va

[153]	valid_0's auc: 0.982091	valid_0's l2: 0.00946649
[154]	valid_0's auc: 0.982121	valid_0's l2: 0.00946225
[155]	valid_0's auc: 0.982162	valid_0's l2: 0.00945536
[156]	valid_0's auc: 0.982196	valid_0's l2: 0.00945437
[157]	valid_0's auc: 0.982224	valid_0's l2: 0.00944956
[158]	valid_0's auc: 0.982265	valid_0's l2: 0.0094422
[159]	valid_0's auc: 0.982251	valid_0's l2: 0.0094459
[160]	valid_0's auc: 0.982272	valid_0's l2: 0.00944472
[161]	valid_0's auc: 0.982308	valid_0's l2: 0.00943867
[162]	valid_0's auc: 0.982363	valid_0's l2: 0.00943571
[163]	valid_0's auc: 0.982377	valid_0's l2: 0.00943242
[164]	valid_0's auc: 0.982407	valid_0's l2: 0.00942926
[165]	valid_0's auc: 0.982413	valid_0's l2: 0.0094244
[166]	valid_0's auc: 0.982464	valid_0's l2: 0.00942099
[167]	valid_0's auc: 0.982465	valid_0's l2: 0.00941901
[168]	valid_0's auc: 0.982495	valid_0's l2: 0.0094145
[169]	valid_0's auc: 0.982518	valid_0's l2: 0.00940971
[170]	valid_0's auc: 0.982526	valid_0's l2: 0.00940519
[171]	valid_0'

In [18]:
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_a.csv")
test_data = pd.read_csv(csv_path)
test_data = test_data.fillna(0)
X_df_pred = test_data.loc[:,'date':'f297']
X_pred = X_df_pred.as_matrix()
y_pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
y_pred[30000:400000]

array([0.000735  , 0.00037143, 0.00045104, ..., 0.00409387, 0.07399518,
       0.00113651])

In [19]:
result = pd.DataFrame({'id':test_data['id'].as_matrix(), 'score':y_pred})
result.to_csv("result2.csv", index=False)